**Následující dvě hypotézy ověříme s 95% jistotou.**

### Hypotéza 1:

*Na silnicích první třídy se byly nehody s následky na zdraví se stejnou pravděpodobností
jako na dálnicích.*

K ověření hypotézy využijte 𝜒2 test s tím, že také určíte, jestli nehody na silnicích 1. třídy
vedly častěji či méně často k nehodě s následky na zdraví (sloupec p9) než na dálnicích
(p36). V tom vám může pomoct “expected” výstup 𝜒2 testu.

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

df = pd.read_pickle('accidents.pkl.gz')

df = df[df['p36'].isin([0, 1])]
contingency_table = pd.crosstab(df['p36'], df['p9'])

#! DEBUG
print(contingency_table)

#! DEBUG
# contingency_table = [[5200, 2800],
#                      [14600, 7400]]

x2, p_value, DF, expected = scipy.stats.chi2_contingency(contingency_table)

print('Výsledek chi-square testu')
print(f'Chi-square statistika: {x2:.2f}')
print(f'P_value: {p_value:.4f}')
print(f'Degrees of Freedom: {DF}')
print('Očekávané hodnoty:')
print(expected)

alpha = 0.05
if p_value < alpha:
    print('\nHypotezu zamitame. Je tu velky rozdil.')
else:
    print('\nHypotezu NEzamitame. Je to zanedbatelne.')


p9      1      2
p36             
0    1247   6674
1    7059  14773
Výsledek chi-square testu
Chi-square statistika: 794.15
P_value: 0.0000
Degrees of Freedom: 1
Očekávané hodnoty:
[[ 2211.26696468  5709.73303532]
 [ 6094.73303532 15737.26696468]]

Hypotezu zamitame. Je tu velky rozdil.


### Hypotéza 2:

*Škoda při nehodách trolejbusů je nižší, než při nehodách autobusů a tato odchylka je
statisticky významná.*

Využijte vhodný test, který vyberete na základě distribuce dat, nevyužívejte 𝜒2 test. Určete,
zda je škoda na vozidlech vyšší či nižší a zda je tato vlastnost na požadované hladině
významnosti.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

df_accidents = pd.read_pickle('accidents.pkl.gz')
df_vehicles = pd.read_pickle('vehicles.pkl.gz')
df = pd.merge(df_accidents, df_vehicles, on='p1', how='inner')

#! DEBUG
# print(df.columns)

# df['p14*100'] skoda v kc
# df['p44'] == 11 trolejbus
# df['p44'] == 8 autobus
trolleybus_damages = df[df['p44'] == 11]['p14*100'].dropna()
bus_damages = df[df['p44'] == 8]['p14*100'].dropna()

print(f"Number of trolleybus entries: {len(trolleybus_damages)}")
print(f"Number of bus entries: {len(bus_damages)}")

# Perform Shapiro-Wilk test for normality
_, p_trolley = scipy.stats.shapiro(trolleybus_damages)
_, p_bus = scipy.stats.shapiro(bus_damages)
print(f"P-value for trolleybus normality: {p_trolley:.4f}")
print(f"P-value for bus normality: {p_bus:.4f}")

# Perform t-test
t_stat, p_value = scipy.stats.ttest_ind(trolleybus_damages, bus_damages, equal_var=False)  # Welch's t-test
print("\nT-test Results")
print(f"T-statistic: {t_stat:.2f}")
print(f"P-value: {p_value:.4f}")

# Perform Mann-Whitney U test
u_statistic, p_value = scipy.stats.mannwhitneyu(trolleybus_damages, bus_damages, alternative='two-sided')
print(f'\nMann-Whitney U Statistic: {u_statistic:.2f}')
print(f'P-value: {p_value:.4f}')

alpha = 0.05
if p_value < alpha:
    print('\nHypotezu zamitame. Je tu velky rozdil.')
else:
    print('\nHypotezu NEzamitame. Je to zanedbatelne.')


Number of trolleybus entries: 293
Number of bus entries: 4284
P-value for trolleybus normality: 0.0000
P-value for bus normality: 0.0000

T-test Results
T-statistic: -5.00
P-value: 0.0000

Mann-Whitney U Statistic: 467959.50
P-value: 0.0000

Hypotezu zamitame. Je tu velky rozdil.
